This script compares the impact of increasing the number of edges in random small world graphs. Since the theoretical motivation behind our approach relies on graphins, we expect that increasing the density of edges will hinder the performance. In addition, our proposed method is compared with other baselines.

In [9]:
import cvxpy as cp
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.covariance import GraphicalLasso

import time
from joblib import Parallel, delayed
from os import cpu_count

import utils
import spectral_nti as snti

# %matplotlib qt

SEED = 0
N_CPUS = cpu_count()
np.random.seed(SEED)

## Auxiliary functions

In [7]:
def run_exp(id, models, KK, iters):
    L_hat = np.zeros((N, N, len(models), len(KK)))
    lamd_hat = np.zeros((N, len(models), len(KK)))
    Ls = np.zeros((N, N, len(KK)))
    lamds = np.zeros((N, len(KK)))
    cs_err = np.zeros((len(models), len(KK)))
    for i, k in enumerate(KK):
        # Create graphs
        A0 = nx.to_numpy_array(nx.watts_strogatz_graph(N0, k, p, seed=SEED))
        L0 = np.diag(np.sum(A0, 0)) - A0
        lambdas0, _ = np.linalg.eigh(L0)

        A = nx.to_numpy_array(nx.watts_strogatz_graph(N, k, p))
        Ls[:,:,i] = np.diag(np.sum(A, 0)) - A
        lamds[:,i], V = np.linalg.eigh(Ls[:,:,i])

        for j, model in enumerate(models):
            if model['name'] in ['MGL-Tr', 'MGL-Sq', 'MGL-Heat', 'MGL-Poly']:
                model['cs'], cs_err[j,i] = utils.compute_cs(model['gs'], lambdas0, lamds[:,i])
            elif model['name'] == 'MGL-Tr=1':
                model['cs'] = 1

        # Create signals
        X = utils.create_signals(Ls[:,:,i], M)
        C_hat = X@X.T/M

        # Estimate graph
        for j, model in enumerate(models):
            t = time.time()
            L_hat[:,:,j,i], lamd_hat[:,j,i] = utils.est_graph(C_hat, model, iters)
            t = time.time() - t
            print('Graph-{}, k-{}: Model: {} - cs: {:.4f} - Time(sec): {:.3}'.
                  format(id, k, model['name'], cs_err[j,i], t))
    return L_hat, lamd_hat, Ls, lamds, cs_err


def est_errs1(Ls, lamds, L_hat, lams_hat, sq_err=True):
    err_lamb = np.zeros(lams_hat.shape[1:])
    err_L = np.zeros(L_hat.shape[2:])
    for k in range(L_hat.shape[-1]):
        for i in range(L_hat.shape[-2]):
            L = Ls[:,:,i,k]
            lamd = lamds[:,i,k]
            norm_L = np.linalg.norm(L, 'fro')
            norm_lam = np.linalg.norm(lamd, 2)

            for j in range(L_hat.shape[-3]):
                err_L[j,i,k] = np.linalg.norm(L-L_hat[:,:,j,i,k], 'fro')/norm_L
                err_lamb[j,i,k] = np.linalg.norm(lamd-lams_hat[:,j,i,k], 2)/norm_lam

                if sq_err:
                    err_L[j,i,k] = err_L[j,i,k]**2
                    err_lamb[j,i,k] = err_lamb[j,i,k]**2

    return err_L, err_lamb


def est_errs2(Ls, lamds, L_hat, lams_hat, sq_err=True):
    err_lamb = np.zeros(lams_hat.shape[1:])
    err_L = np.zeros(L_hat.shape[2:])
    for k in range(L_hat.shape[-1]):
        for i in range(L_hat.shape[-2]):
            L = Ls[:,:,i,k]
            lamd = lamds[:,i,k]
            norm_L = np.linalg.norm(L, 'fro')
            norm_lam = np.linalg.norm(lamd, 2)

            for j in range(L_hat.shape[-3]):
                if np.all((L_hat[:,:,j,i,k] == 0)):
                    norm_L_hat = 1
                    norm_lam_hat = 1
                else:
                    norm_L_hat = np.linalg.norm(L_hat[:,:,j,i,k], 'fro')
                    norm_lam_hat = np.linalg.norm(lams_hat[:,j,i,k], 2)

                L_hat_norm = L_hat[:,:,j,i,k]/norm_L_hat
                lam_hat_norm = lams_hat[:,j,i,k]/norm_lam_hat
                err_L[j,i,k] = np.linalg.norm(L/norm_L-L_hat_norm,'fro')
                err_lamb[j,i,k] = np.linalg.norm(lamd/norm_lam-lam_hat_norm, 2)

                if sq_err:
                    err_L[j,i,k] = err_L[j,i,k]**2
                    err_lamb[j,i,k] = err_lamb[j,i,k]**2

    return err_L, err_lamb


def plot_err(KK, models, err, ylab, semlogy=True, ylim=[]):
    plt.figure()
    for i, model in enumerate(models):
        if semlogy:
            plt.semilogy(KK, err[i,:], model['fmt'], label=model['name'],
                        linewidth=2, markersize=12)
        else:
            plt.plot(KK, err[i,:], model['fmt'], label=model['name'],
                     linewidth=2, markersize=12)
        plt.grid(True)
        plt.xlabel('Mean node degree')
        plt.ylabel(ylab)
        plt.legend()
        plt.xlim([KK[0], KK[-1]])
        if ylim:
            plt.ylim(ylim)
        plt.tight_layout()

## Constants

In [3]:
# Graph params
p = .1
N0 = 150
N = 100
M = 500

n_graphs = 50
KK = [8, 12, 16, 20, 24, 28, 32, 36, 40]
iters = 200

GS = [
    lambda a, b : cp.sum(a)/b,    # delta: 4e-2
    lambda a, b : cp.sum(a**2)/b,  # delta: .7
    lambda a, b : cp.sum(cp.exp(-a))/b,    # delta: 3e-3
    lambda a, b : cp.sum(.25*a**2-.75*a)/b,
]
BOUNDS = [
    lambda lamd, lamd_t, b : -2/b*lamd_t.T@lamd,
    lambda lamd, lamd_t, b : 1/b*cp.exp(-lamd_t).T@lamd,
    lambda lamd, lamd_t, b: 1/b*(0.75-2*0.25*lamd_t).T@lamd,
]

deltas = [1e-3, .3, .005, .1]

models = [
    # Baselines
    {'name': 'GLasso', 'alpha': 0.01, 'fmt': '^-'},
    {'name': 'SGL', 'fmt': '*--', 'regs': {'alpha': 0, 'beta': 25, 'c1': .1, 'c2': 30, 'conn_comp': 1}},
    {'name': 'MGL-Tr=1', 'gs': GS[0], 'bounds': [], 'fmt': '2--',
     'regs': {'alpha': 0, 'beta': .9, 'gamma': 0, 'deltas': deltas[0]}},

    # Our algorithms
    {'name': 'MGL-Tr', 'gs': GS[0], 'bounds': [], 'fmt': '2-',
     'regs': {'alpha': 0, 'beta': 5, 'gamma': 0, 'deltas': deltas[0]}},
    {'name': 'MGL-Sq', 'gs': GS[1], 'bounds': BOUNDS[0], 'fmt': 'o-',
     'regs': {'alpha': 0, 'beta': 25, 'gamma': 50, 'deltas': deltas[1]}},
    {'name': 'MGL-Heat', 'gs': GS[2], 'bounds': BOUNDS[1], 'fmt': 'x-',
     'regs': {'alpha': 0, 'beta': 10, 'gamma': 100, 'deltas': deltas[2]}},
]

## Compare contraints

In [4]:
# Estimate graph
total_t = time.time()
Ls = np.zeros((N, N, len(KK), n_graphs))
lamds = np.zeros((N, len(KK), n_graphs))
Ls_hat = np.zeros((N, N, len(models), len(KK), n_graphs))
lamds_hat = np.zeros((N, len(models), len(KK), n_graphs))
cs_err = np.zeros((len(models), len(KK), n_graphs))
print('N_CPUS:', N_CPUS)
pool = Parallel(n_jobs=N_CPUS, verbose=0)
resps = pool(delayed(run_exp)(i, models, KK, iters) for i in range(n_graphs))

for i, resp in enumerate(resps):
    Ls_hat[:,:,:,:,i], lamds_hat[:,:,:,i], Ls[:,:,:,i], \
        lamds[:,:,i], cs_err[:,:,i] = resp

total_t = time.time() - total_t
print('-----', total_t/60, ' mins -----')

N_CPUS: 12
----- 60.227008028825125  mins -----


In [18]:
err_L, err_lamb = est_errs2(Ls, lamds, Ls_hat, lamds_hat)
# mean_cs_err = np.abs(np.mean(cs_err, 2))
mean_err_L = np.median(err_L, 2)
mean_err_lam = np.mean(err_lamb, 2)
# plot_err(KK, models, mean_cs_err, 'Mean cs err', semlogy=True)
plot_err(KK, models, mean_err_L, 'Mean Err2 L')
plot_err(KK, models, mean_err_lam, 'Mean Err2 lambdas')

In [14]:
err_L, _ = est_errs1(Ls, lamds, Ls_hat, lamds_hat)
mean_err_L = np.median(err_L, 2)
plot_err(KK, models, mean_err_L, 'Mean Err1 L')

In [6]:
# # Save data
# models_aux = [{'name': model['name'], 'fmt': model['fmt'], 'regs': model['regs']}
#                 for model in models]
# data = {
#     'Ls': Ls,
#     'lamds': lamds,
#     'Ls_hat': Ls_hat,
#     'lamds_hat': lamds_hat,
#     'cs_err': cs_err,
#     'KK': KK,
#     'iters' : iters,
#     'models': models_aux
# }
# file = 'density_{}graphs_{}iters'.format(n_graphs, iters)
# np.save('results/graph_density/' + file, data)

In [17]:
# # Load data
%matplotlib qt
file = 'results/graph_density/density_50graphs_200iters_weighted.npy'
data = np.load(file, allow_pickle=True).item()
models = data['models']

Ls = data['Ls']
lamds = data['lamds']
Ls_hat = data['Ls_hat']
lamds_hat = data['lamds_hat']
KK = data['KK']
iters = data['iters']
models = data['models']